# imports

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import os
from tensorflow.keras.applications.resnet50 import preprocess_input

2025-07-30 11:32:38.932308: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-30 11:32:38.942651: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 11:32:39.020488: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 11:32:39.094906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753862559.156374   19698 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753862559.17

# pathes

In [ ]:
train_path = '/kaggle/input/big-dog-and-cat-dataset/'
test_path = '/kaggle/input/dog-and-cat/data/'

# import data

In [ ]:
file_list1 = [f for f in os.listdir(train_path + 'train/')]
df1 = pd.DataFrame({'filename': file_list1})
df1['label'] = df1['filename'].apply(lambda x: x.split('.')[0])
print(df1.head())

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/big-dog-and-cat-dataset/train/'

# data generator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8,1.2]
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train = train_datagen.flow_from_dataframe(
    dataframe=df1,
    directory='/kaggle/input/big-dog-and-cat-dataset/train/', 
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True 
)

validation = val_datagen.flow_from_directory(
    test_path + 'test',
    target_size=(224, 224),
    batch_size=128,
    class_mode='binary'
)
test = test_datagen.flow_from_directory(
    test_path + 'validation',
    target_size=(224, 224),
    batch_size=128,
    class_mode='binary'
)

In [ ]:
images, labels = next(train)
num_images = 8
plt.figure(figsize=(15, 5))
for i in range(num_images):
    plt.subplot(2, 4, i + 1)
    plt.imshow(images[i])
    label = 'dog' if labels[i] == 1 else 'cat'
    plt.title(f'Label: {label}')
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
from tensorflow.keras.applications import ResNet50

2025-07-30 11:45:40.023578: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-30 11:45:40.023937: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 11:45:40.026026: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 11:45:40.031846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753863340.041607   20857 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753863340.04

# model

In [ ]:
input_tensor = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)
for layer in base_model.layers:
    layer.trainable = False
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output1 = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output1)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(train, validation_data=validation, epochs=15)


2025-07-30 11:45:43.901606: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5: 403 -- Forbidden

# fine-tune

In [ ]:
for layer in base_model.layers[-50:]:
    layer.trainable = True
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(train, validation_data=validation, epochs=5)

# accuracy

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')
plt.grid(True)
plt.show()

# loss

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs Validation Loss')
plt.grid(True)
plt.show()

# test_model

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy:.4f}")